In [127]:
%%capture
# Importing required libraries
import pandas as pd
import numpy as np

#from pandasql import sqldf
#Setting up a funtion to pass global settings with sqldf queries
#pysqldf = lambda q: sqldf(q, globals())


In [128]:
def ColumnNameStd(df_name):
  df_name.columns = df_name.columns.str.replace(' ', '_') # replacing spaces (' ') with  underscore.
  df_name.columns = df_name.columns.str.replace('-', '_') # replacing hyphens ('-') with underscore
  df_name.columns = df_name.columns.str.replace('[', '')
  df_name.columns = df_name.columns.str.replace(']', '')
  df_name.columns = df_name.columns.str.lower()  
  return df_name

In [145]:
DevelopedCountries = ['australia', 'austria','germany', 'usa','united states of america', 'norway', 'japan', 'canada',
                      'poland', 'croatia', 'hungary', 'greece', 'romania','slovakia','italy','belgium','spain','united kingdom','france','latvia']

DevelopingCountries = ['brazil', 'mexico', 'argentina', 'peru', 
                       'india', 'bangladesh', 'nepal','myanmar'
                       'oman', 'qatar', 'lebanon', 'turkey', 'jordan', 
                       'south africa', 'namibia', 'zimbabwe', 'morocco', 'ehiopia', 'eritrea', 'kenya'
                       'malaysia', 'republic of korea', 'thailand', 'indonesia', 'philippines']

# World Economic Indicator Data

## Metric 1

In [206]:
wld_gdp = pd.read_csv('https://raw.githubusercontent.com/pruthvireddy/SDGGoals_ACausalStudy/main/Data/Raw/WLD_GDP.csv')
wld_gdp = ColumnNameStd(wld_gdp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [207]:
wld_gdp = wld_gdp[wld_gdp['series_name'].notna()]

wld_gdp = wld_gdp.replace('..', np.nan)

int_columns = ['2000_yr2000', '2001_yr2001', '2002_yr2002', '2003_yr2003',
       '2004_yr2004', '2005_yr2005', '2013_yr2013', '2014_yr2014',
       '2015_yr2015', '2016_yr2016', '2017_yr2017', '2018_yr2018',
       '2019_yr2019', '2020_yr2020', '2021_yr2021']

wld_gdp[int_columns] = wld_gdp[int_columns].astype(float)

wld_gdp = pd.melt(wld_gdp, id_vars=['country_name', 'country_code', 'series_name', 'series_code'], var_name='year', value_name='value')

wld_gdp['year'] = wld_gdp['year'].str.split('_', expand=True)[0]

In [208]:
conditions = [
    (wld_gdp['series_name'] == 'GDP (constant 2015 US$)') ,
    (wld_gdp['series_name'] == 'GDP (current US$)'),
    (wld_gdp['series_name'] == 'GDP per capita (constant 2015 US$)') ,
    (wld_gdp['series_name'] == 'GDP per capita, PPP (current international $)') ,
    (wld_gdp['series_name'] == 'GDP per capita, PPP (constant 2017 international $)') ,
    (wld_gdp['series_name'] == 'GDP per person employed (constant 2017 PPP $)') ,
    (wld_gdp['series_name'] == 'GDP, PPP (current international $)'),
    (wld_gdp['series_name'] == 'Manufacturing, value added (% of GDP)')]

choices = ['GDP_15', 'GDP','GDP_PC','GDP_PC_PPP','GDP_PC_PPP_17','GDP_PEMP','GDP_PPP','MFG_VA']
wld_gdp['series_name'] = np.select(conditions, choices, default=np.nan)
wld_gdp = wld_gdp.drop('series_code', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


In [209]:
wld_gdp = wld_gdp.pivot_table(index=['country_name', 'country_code','year'], values='value', columns='series_name') \
    .reset_index() \
    .rename_axis(None, axis=1)

wld_gdp['metric_key'] = wld_gdp["country_name"].str.lower() + '-' + wld_gdp["year"].astype(str)

In [210]:
conditions = [
    (wld_gdp['country_name'].str.lower().isin(DevelopedCountries)) ,
    
    (wld_gdp['country_name'].str.lower().isin(DevelopingCountries))
    ]

choices = ['Developed', 'Developing']
wld_gdp['economic_category'] = np.select(conditions, choices, default= 'Others')

wld_gdp['metric_key'] = wld_gdp["economic_category"].str.lower() + '-' + wld_gdp["year"].astype(str)

In [211]:
wld_gdp

,country_name,country_code,year,GDP,GDP_15,GDP_PC,GDP_PC_PPP,GDP_PC_PPP_17,GDP_PEMP,GDP_PPP,MFG_VA,metric_key,economic_category
0,Afghanistan,AFG,2002,4.055180e+09,7.228792e+09,319.847107,877.014268,1189.784668,5572.184800,1.982120e+10,18.822752,others-2002,Others
1,Afghanistan,AFG,2003,4.515559e+09,7.867259e+09,332.220010,927.857548,1235.810063,5742.153718,2.197247e+10,16.923866,others-2003,Others
2,Afghanistan,AFG,2004,5.226779e+09,7.978512e+09,322.668009,925.441429,1200.278013,5529.232361,2.288310e+10,17.554006,others-2004,Others
3,Afghanistan,AFG,2005,6.209138e+09,8.874476e+09,345.925814,1023.051749,1286.793659,5900.214139,2.624565e+10,16.598212,others-2005,Others
4,Afghanistan,AFG,2013,2.014640e+10,1.836025e+10,568.964544,2015.514775,2116.465258,9451.821169,6.503984e+10,11.499867,others-2013,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,Zimbabwe,ZWE,2018,1.811554e+10,2.207732e+10,1529.026374,4017.221716,3923.030495,8464.795850,5.800391e+10,7.721094,developing-2018,Developing
2877,Zimbabwe,ZWE,2019,1.928429e+10,2.072084e+10,1414.829099,3783.547898,3630.033985,7806.484516,5.541185e+10,15.738234,developing-2019,Developing
2878,Zimbabwe,ZWE,2020,1.805117e+10,1.942605e+10,1307.013630,3537.351394,3353.411304,7309.497926,5.257540e+10,18.430002,developing-2020,Developing
2879,Zimbabwe,ZWE,2000,6.689958e+09,2.015994e+10,1696.753262,2272.599366,4353.368199,9624.277338,2.700185e+10,13.387982,developing-2000,Developing


In [212]:
wld_gdp.columns

Index(['country_name', 'country_code', 'year', 'GDP', 'GDP_15', 'GDP_PC',
       'GDP_PC_PPP', 'GDP_PC_PPP_17', 'GDP_PEMP', 'GDP_PPP', 'MFG_VA',
       'metric_key', 'economic_category'],
      dtype='object')

In [213]:
wld_gdp = wld_gdp.groupby(['metric_key','year', 'economic_category'])['GDP', 'GDP_15', 'GDP_PC','GDP_PC_PPP', 'GDP_PC_PPP_17', 'GDP_PEMP', 'GDP_PPP', 'MFG_VA'].mean().round(2)
wld_gdp = pd.DataFrame(wld_gdp).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


## Metric 2

In [196]:
EcityAcc = pd.read_csv('https://raw.githubusercontent.com/pruthvireddy/SDGGoals_ACausalStudy/main/Data/Raw/ECITY_ACC_Panel.csv')
EcityAcc = ColumnNameStd(EcityAcc)

EcityAcc = EcityAcc[EcityAcc['series_name'].notna()]

EcityAcc = EcityAcc.replace('..', np.nan)

int_columns = ['2000_yr2000', '2001_yr2001', '2002_yr2002', '2003_yr2003',
       '2004_yr2004', '2005_yr2005', '2013_yr2013', '2014_yr2014',
       '2015_yr2015', '2016_yr2016', '2017_yr2017', '2018_yr2018',
       '2019_yr2019', '2020_yr2020', '2021_yr2021']

EcityAcc[int_columns] = EcityAcc[int_columns].astype(float)

EcityAcc = pd.melt(EcityAcc, id_vars=['country_name', 'country_code', 'series_name', 'series_code'], var_name='year', value_name='value')

EcityAcc['year'] = EcityAcc['year'].str.split('_', expand=True)[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [198]:
conditions = [
    (EcityAcc['series_name'] == 'Access to electricity (% of population)') ,
    (EcityAcc['series_name'] == 'Access to electricity, rural (% of rural population)')
    ]

choices = ['ELC_POP', 'ELC_RUR_POP']
EcityAcc['series_name'] = np.select(conditions, choices, default=np.nan)
EcityAcc = EcityAcc.drop('series_code', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [200]:
EcityAcc = EcityAcc.pivot_table(index=['country_name', 'country_code','year'], values='value', columns='series_name') \
    .reset_index() \
    .rename_axis(None, axis=1)

In [202]:
conditions = [
    (EcityAcc['country_name'].str.lower().isin(DevelopedCountries)) ,
    
    (EcityAcc['country_name'].str.lower().isin(DevelopingCountries))
    ]

choices = ['Developed', 'Developing']
EcityAcc['economic_category'] = np.select(conditions, choices, default= 'Others')

EcityAcc['metric_key'] = EcityAcc["economic_category"].str.lower() + '-' + EcityAcc["year"].astype(str)

In [203]:
EcityAcc = EcityAcc.groupby(['metric_key','year', 'economic_category'])['ELC_POP', 'ELC_RUR_POP'].mean()
EcityAcc = pd.DataFrame(EcityAcc).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


# SDG Metrics

In [171]:
CO2GDP = pd.read_csv('https://raw.githubusercontent.com/pruthvireddy/SDGGoals_ACausalStudy/main/Data/Raw/ATM_CO2_GDP_table.csv')
CO2GDP = ColumnNameStd(CO2GDP)
CO2GDP = CO2GDP[['goal', 'target', 'indicator', 'geoareaname', 'timeperiod', 'value', 'units']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [172]:
conditions = [
    (CO2GDP['geoareaname'].str.lower().isin(DevelopedCountries)) ,
    
    (CO2GDP['geoareaname'].str.lower().isin(DevelopingCountries))
    ]

choices = ['Developed', 'Developing']
CO2GDP['economic_category'] = np.select(conditions, choices, default= 'Others')

In [173]:
CO2GDP['metric_key'] = CO2GDP["economic_category"].str.lower() + '-' + CO2GDP["timeperiod"].astype(str)
CO2GDP = CO2GDP.groupby(['metric_key','timeperiod', 'economic_category'])['value'].mean()
CO2GDP = pd.DataFrame(CO2GDP).reset_index()

In [175]:
CO2GDP.columns = ['metric_key', 'year', 'economic_category', 'CO2GDP']

In [178]:
AtmCO2 = pd.read_csv('https://raw.githubusercontent.com/pruthvireddy/SDGGoals_ACausalStudy/main/Data/Raw/ATM_CO2_MVA_table.csv')
AtmCO2 = ColumnNameStd(AtmCO2)
AtmCO2 = AtmCO2[['goal', 'target', 'indicator', 'geoareaname', 'timeperiod', 'value', 'units']]
AtmCO2['metric_key'] = AtmCO2["geoareaname"].str.lower() + '-' + AtmCO2["timeperiod"].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [179]:
conditions = [
    (AtmCO2['geoareaname'].str.lower().isin(DevelopedCountries)) ,
    
    (AtmCO2['geoareaname'].str.lower().isin(DevelopingCountries))
    ]

choices = ['Developed', 'Developing']
AtmCO2['economic_category'] = np.select(conditions, choices, default= 'Others')

In [180]:
AtmCO2['metric_key'] = AtmCO2["economic_category"].str.lower() + '-' + AtmCO2["timeperiod"].astype(str)
AtmCO2 = AtmCO2.groupby(['metric_key','timeperiod', 'economic_category'])['value'].mean()
AtmCO2 = pd.DataFrame(AtmCO2).reset_index()

In [181]:
AtmCO2.columns = ['metric_key', 'year', 'economic_category', 'AtmCO2']

# Final Join


In [229]:
SDGMetrics = pd.merge(CO2GDP,AtmCO2, on = ['metric_key', 'year', 'economic_category'], how = 'inner')

In [230]:
EconInd = pd.merge(wld_gdp,EcityAcc, on = ['metric_key', 'year', 'economic_category'], how = 'inner')

In [231]:
EconInd['year']=EconInd['year'].astype(int)
SDGMetrics['year']=SDGMetrics['year'].astype(int)

In [232]:
GlobalData = pd.merge(EconInd,SDGMetrics, on = ['metric_key', 'year', 'economic_category'], how = 'right')

In [228]:
print(len(CO2GDP))
print(len(AtmCO2))
print(len(wld_gdp))
print(len(EcityAcc))


60
60
42
42


In [234]:
GlobalData.to_csv('GlobalData.csv',index=False)